In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

df = pd.read_csv('norm.csv')

In [2]:
X = df[['PE1', 'PE2', 'PE3', 'PE4', 'TP1', 'TP2']]
y = df['FM1']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
# Преобразование данных в тензоры
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1, 1)  # Меняем форму, чтобы было (n, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float).view(-1, 1)  # Меняем форму, чтобы было (n, 1)

In [5]:
# Создание DataLoader для обучающего и тестового наборов
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [6]:
# Пример создания DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
import torch.nn as nn

class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [ ]:
# Определение модели, оптимизатора и функции потерь
model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()


# Обучение модели
EPOCHS = 20
for epoch in range(EPOCHS):
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Обнуляем градиенты
        outputs = model(inputs)  # Предсказание модели
        loss = loss_fn(outputs, targets)  # Вычисление потерь
        loss.backward()  # Обратное распространение ошибки
        optimizer.step()  # Шаг оптимизатора
    print(f'Эпоха [{epoch+1}/{EPOCHS}], Потери: {loss.item()}')